# Installing Dependencies

In [41]:
using DelimitedFiles
using ProgressMeter
using CUDA
using BenchmarkTools: @btime
include("../../../src/forces.jl")
include("../../../src/neighbor.jl")

cu_knn (generic function with 1 method)

# Initial Conditions

In [43]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
t_f = 5
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 12

# Forces Conditions
r_max = 2.8
fp = 0.002
K = 0.05
s = 2.0

# Coding C0nditions
n_text = 50
num_agg = 2

@time X = Float32.(readdlm("../../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

  0.002353 seconds (12.02 k allocations: 406.844 KiB)


746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

# Using kNN

In [45]:
# Inizializate Variables for kNN
i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu;

In [48]:
# Inizializate Variables for Forces
r_p = zeros(size(X)) |> cu
dist_p = zeros(size(X,1)) |> cu
r = zeros(nn,size(X)[1],3) |> cu
dist = zeros(nn, size(X)[1]) |> cu
F = zeros(nn, size(X)[1],3) |> cu
dX = zeros(size(X)[1],3) |> cu;

In [49]:
t = 0
cu_knn()
cu_forces(t, r_max, fp, K)
X = X + dX*dt;

746×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 -1.4999   -2.88978   -8.98002
  0.4999   -2.88978   -8.98002
  2.5001   -2.88978   -8.98002
 -2.49999  -1.15015   -8.97986
 -0.4998   -1.15006   -8.98002
  1.5002   -1.15006   -8.98002
  3.49989  -1.15021   -8.98002
 -3.50001   0.579874  -8.97986
 -1.4999    0.580164  -8.98002
  0.5001    0.580164  -8.98002
  2.5001    0.580164  -8.98002
 -2.4998    2.31      -8.98001
 -0.4999    2.31005   -8.97984
  ⋮                   
  3.5      -2.31016    8.97989
 -3.50001  -0.579874   8.97986
 -1.4999   -0.579817   8.98002
  0.5001   -0.579817   8.98002
  2.5001   -0.579817   8.98002
 -2.49999   1.15015    8.97986
 -0.4998    1.15006    8.98002
  1.5002    1.15006    8.98002
  3.49989   1.15021    8.98002
 -1.4999    2.88978    8.98002
  0.4999    2.88978    8.98002
  2.5001    2.88978    8.98002

: 